## Tranformar datase de IDS para utilizar apenas dimensão economica

In [ ]:
import pandas as pd
from lxml import html
import requests
import zipfile
import io
import os
from pathlib import Path
import csv

In [ ]:
df = pd.read_csv("http://ftp.dadosabertos.ans.gov.br/FTP/PDA/historico_idss/historico_idss.csv",sep=";",encoding="ISO-8859-1")

In [ ]:
df.columns

In [ ]:
df = df[['Registro','Razao_Social','Modalidade','IDEF_2012_2011','IDEF_2013_2012','IDEF_2014_2013','IDEF_2015_2014','IDSM_2016_2015','IDSM_2017_2016','IDSM_2018_2017']]

## Filtrando para obter apenas cooperativas

In [ ]:
df_cooperativas = df[df['Modalidade'] == "Cooperativa Médica"]

In [ ]:
df_cooperativas.dropna(inplace=True)

In [ ]:
df_cooperativas.head()

In [ ]:
def converter_para_float(x):
    try:
        return float(x.replace(".","").replace(",","."))
    except:
        return 0
        

### Tranformando índices em variáveis do tipo float

In [ ]:
colunas_indices = ['IDEF_2012_2011','IDEF_2013_2012','IDEF_2014_2013','IDEF_2015_2014','IDSM_2016_2015','IDSM_2017_2016','IDSM_2018_2017']
for coluna in colunas_indices:
    df_cooperativas[coluna] = df_cooperativas[coluna].apply(lambda x: converter_para_float(x))

In [ ]:
df_cooperativas.head()

In [ ]:
def converter_para_binario(x):
    for coluna in colunas_indices:
        valor_coluna = coluna.split("_")[-1]
#         print(coluna,valor_coluna)
        if (x[coluna] >= 0.8):
            x["utilizar_"+valor_coluna] = 1
        else:
            x["utilizar_"+valor_coluna] = 0
    return x

In [ ]:
df_cooperativas= df_cooperativas.apply(lambda row: converter_para_binario(row) , axis=1)

In [ ]:
df_cooperativas.drop(columns=colunas_indices,inplace=True)

In [ ]:
df_cooperativas.columns

In [ ]:
df_cooperativas.head()

In [ ]:
df_cooperativas.set_index("Registro",inplace=True)

In [ ]:
df_porte = pd.read_csv("../entrada/porte_operadoras.csv",encoding="ISO-8859-1")
df_porte.drop(columns=['beneficiarios'],inplace=True)

In [ ]:
df_porte["alto_nivel"] = 0

In [ ]:
def marcar_registros_utilizacao(x):
    _ano = x['ano']
    valor = 0
    if (_ano in range(2011,2018)):
        nome_coluna = "utilizar_"+str(_ano)
        try:
            valor =  df_cooperativas.loc[x['Registro_ANS'],:][nome_coluna]
        except:
            pass
    return valor

In [ ]:
df_porte['alto_nivel'] = df_porte.apply(lambda x: marcar_registros_utilizacao(x),axis=1)

In [ ]:
if (not(os.path.exists('../entrada'))):
    os.mkdir('../entrada')

In [ ]:
len(df_porte)

In [ ]:
df_porte = df_porte[df_porte['alto_nivel'] == 1]

In [ ]:
len(df_porte)

In [ ]:
df_porte.to_csv("../entrada/idss_alto_nivel.csv",index=False,encoding="ISO-8859-1")

In [ ]:
df_porte.groupby(['ano','porte'])['Registro_ANS'].count()